In [1]:
from LSTM import LSTM
import torch
import numpy as np 
from torch import nn
from sklearn.preprocessing import MinMaxScaler
from torch.autograd import Variable


In [2]:
################
#  DATALOADING #
################

In [46]:

def sliding_windows(data,seq_length):
    x = []
    y = []

    for i in range(data.shape[1]-seq_length-1):
        _x = data[:,i:(i+seq_length)]
        _y = data[:,i+seq_length: i + 2*seq_length]
        if _y.shape[1] != seq_length:
            return np.array(x),np.array(y)
        x.append(_x)
        y.append(_y)

    return np.array(x),np.array(y)





In [47]:

data = np.load('midprices-new.npy')
#Reshaping data and seperating training and test set
sc = MinMaxScaler()
data = sc.fit_transform(data.reshape(-1,1))
data = data.reshape((1,data.shape[0]))
seq_length = 4
x, y = sliding_windows(data, seq_length)

bsz = 64
size = y.shape[0]
train_size = int(size * 0.67)
test_size = size - train_size

dataX = Variable(torch.Tensor(np.array(x)))
dataY = Variable(torch.Tensor(np.array(y)))


trainX = dataX[0:train_size,:,:]
trainY = dataY[0:train_size,:,:]


testX = dataX[train_size:size, :,:]
testY = dataY[train_size:size,:,:]

trainingset =[]
for i in range(len(trainY)):
    trainingset.append((trainX[i,:,:], trainY[i,:,:]))


trainloader = torch.utils.data.DataLoader(trainingset, batch_size=bsz, shuffle=False)


In [50]:
num_epochs = 2000
learning_rate = 1e-2

input_size = 4
hidden_size = 256
num_layers = 1
seq_length = 4
num_classes = 4

lstm = LSTM(num_classes, input_size, hidden_size, num_layers, seq_length, fc1_out = 128)



In [69]:

test_hist = []
criterion = torch.nn.MSELoss()    # mean-squared error for regression
optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    total_loss = 0
    j = 0  
    for i, (inputs, targets) in enumerate(trainloader):
        outputs = lstm(inputs.to(device))
        outputs = outputs.unsqueeze(1)
        optimizer.zero_grad()
    # obtain the loss function
        loss = criterion(outputs, targets.to(device))
    
        loss.backward()
    
        optimizer.step()
        
        total_loss += loss.item()
        j +=1
    if testX is not None:
        with torch.no_grad():
            y_test_pred = lstm(testX.to(device))
            y_test_pred = y_test_pred.unsqueeze(1)
            test_loss = criterion(y_test_pred.float(), testY.to(device))
            test_hist.append( test_loss.item())
    total_loss /=j
    if epoch % 2 == 0:
        print("Epoch: %d, train loss: %1.5f, test loss: %1.5f" % (epoch, total_loss, test_hist[epoch]))
        torch.save(lstm.state_dict(), f'../Models/seq-regression')

Epoch: 0, train loss: 0.00128, test loss: 0.00110
Epoch: 2, train loss: 0.00130, test loss: 0.00097
Epoch: 4, train loss: 0.00129, test loss: 0.00111
Epoch: 6, train loss: 0.00135, test loss: 0.00102
Epoch: 8, train loss: 0.00131, test loss: 0.00101
Epoch: 10, train loss: 0.00134, test loss: 0.00114
Epoch: 12, train loss: 0.00129, test loss: 0.00104
Epoch: 14, train loss: 0.00133, test loss: 0.00085
Epoch: 16, train loss: 0.00136, test loss: 0.00103
Epoch: 18, train loss: 0.00131, test loss: 0.00094
Epoch: 20, train loss: 0.00132, test loss: 0.00097
Epoch: 22, train loss: 0.00131, test loss: 0.00090
Epoch: 24, train loss: 0.00131, test loss: 0.00089
Epoch: 26, train loss: 0.00128, test loss: 0.00088
Epoch: 28, train loss: 0.00134, test loss: 0.00101
Epoch: 30, train loss: 0.00131, test loss: 0.00115
Epoch: 32, train loss: 0.00134, test loss: 0.00103
Epoch: 34, train loss: 0.00128, test loss: 0.00126
Epoch: 36, train loss: 0.00130, test loss: 0.00139
Epoch: 38, train loss: 0.00132, test

KeyboardInterrupt: 

In [79]:
import matplotlib.pyplot as plt

lstm.eval()
index = np.random.randint(testX.size(0))
print(index)
truth = sc.inverse_transform(testY[index,:,:].squeeze(1))
prediction = lstm(testX[index,:,:].unsqueeze(0)).detach().numpy()
prediction = sc.inverse_transform(prediction.squeeze(0).reshape(1,-1))

print(truth,prediction)

16467
[[249.50000933 268.0000101  234.50000365 262.50000595]] [[248.22728 248.4066  248.87589 249.74606]]


In [13]:
import pickle
with open('sc_midprice', 'wb') as f:
    pickle.dump(sc, f)
    